In [5]:
from imports import *
from utils import *
from model import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data loading

dataset = 'ECG'

data_dir = 'raw\ECG'
train_dataset, test_dataset = load_ecg_data(data_dir)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
# Model Definition

num_classes = 2

model = NET_1k_ECG(num_classes).to(device)

total_weights = count_weights(model)
print(f"The network has {total_weights} weights that need to be deployed.")

model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
model = torch.quantization.prepare_qat(model, inplace=True)
model = torch.quantization.convert(model, inplace=True)

The network has 112 weights that need to be deployed.


In [7]:
# Load Model

# Specify the filename of the saved model
model_filename = f"checkpoint/ECG-112-99.20_model.t7"

# Load the checkpoint from the file, mapping the model to the specified device
checkpoint = torch.load(model_filename, map_location=device)
model.load_state_dict(checkpoint['model'])

print(model) # The model has been quantized to int8

NET_1k_ECG(
  (quant): Quantize(scale=tensor([0.0371]), zero_point=tensor([46]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (conv1_depthwise): QuantizedConv1d(1, 1, kernel_size=(7,), stride=(1,), scale=0.039918337017297745, zero_point=58, padding=(4,), dilation=(2,))
  (conv1_pointwise): QuantizedConv1d(1, 5, kernel_size=(1,), stride=(1,), scale=0.037377890199422836, zero_point=79)
  (bn1): QuantizedBatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation1): QuantizedHardswish()
  (conv2_depthwise): QuantizedConv1d(5, 5, kernel_size=(5,), stride=(1,), scale=0.09448608756065369, zero_point=76, padding=(1,), groups=5)
  (conv2_pointwise): QuantizedConv1d(5, 5, kernel_size=(1,), stride=(1,), scale=0.2084026336669922, zero_point=64)
  (activation2): QuantizedHardswish()
  (conv3_depthwise): QuantizedConv1d(5, 5, kernel_size=(3,), stride=(1,), scale=0.1903160959482193, zero_point=57, padding=(1,), groups=5)
  (conv3_pointwise): QuantizedConv1d(5, 5, 

In [8]:
# Test the model on the test dataset and store the accuracy
model.eval()
test_accuracy = test_model(model, test_loader)

# Extract the int8 weights from all quantized layers in the model and save them to a list
quantized_weights_list = print_quantized_weights(model)

print(f"The test accuracy after quantization is: {test_accuracy:.2f}%")

Extracting weights from layer: conv1_depthwise
Weight Shape: (7,), Int8 Weights:
[-23 -41 127  -7  23  -5  20]

Extracting weights from layer: conv1_pointwise
Weight Shape: (5,), Int8 Weights:
[ 127 -128 -128 -128 -127]

Extracting weights from layer: conv2_depthwise
Weight Shape: (25,), Int8 Weights:
[ -83 -128 -102  110   70 -128   14   39   42  -16   31   28 -128  -39
  -44 -128    3   50  -54   76   29  127 -122  -34   18]

Extracting weights from layer: conv2_pointwise
Weight Shape: (25,), Int8 Weights:
[-128   21  -73  -92  -15   36   74   13  127  -62   51  -12  -68  -32
 -127   24  127  -48   37  -89 -113  -33  -15 -128   51]

Extracting weights from layer: conv3_depthwise
Weight Shape: (15,), Int8 Weights:
[-119 -127   48  -16  127   50 -102  127  -66  127  110   89   53  127
   62]

Extracting weights from layer: conv3_pointwise
Weight Shape: (25,), Int8 Weights:
[ -22  -83 -128   99  -23   59  -50    7  -24 -128   -7  -50   29  127
  -92  -85 -101   37 -128   63   -5   56  1